<a href="https://colab.research.google.com/github/Jigyass/Data-Privacy-and-Data-Security-Models/blob/main/l_diversity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip /content/drive/MyDrive/Data_Privacy_and_Data_Security/adult.zip

Archive:  /content/drive/MyDrive/Data_Privacy_and_Data_Security/adult.zip
  inflating: Index                   
  inflating: adult.data              
  inflating: adult.names             
  inflating: adult.test              
  inflating: old.adult.names         


In [3]:
import pandas as pd

#Read the adult.data file as a pandas dataframe

df = pd.read_csv("adult.data")

In [5]:
df.columns = ['Age', '2', '3', 'Education', '5', 'Marital-Status', 'Occupation', '8', 'Race', '10', '11', '12', '13', '14', '15']

In [7]:
unique_values_occupation = df['Occupation'].unique()
print(unique_values_occupation)

[' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Adm-clerical' ' Sales' ' Craft-repair'
 ' Transport-moving' ' Farming-fishing' ' Machine-op-inspct'
 ' Tech-support' ' ?' ' Protective-serv' ' Armed-Forces'
 ' Priv-house-serv']


In [8]:
def generalize_occupation_level1(occupation):
    occupation_mapping = {
        ' Exec-managerial': 'Management',
        ' Prof-specialty': 'Professional',
        ' Other-service': 'Service',
        ' Adm-clerical': 'Clerical',
        ' Sales': 'Sales',
        ' Craft-repair': 'Crafts',
        ' Transport-moving': 'Transport',
        ' Farming-fishing': 'Farming',
        ' Machine-op-inspct': 'Machine Operation',
        ' Tech-support': 'Tech Support',
        ' ?': 'Unknown',
        ' Protective-serv': 'Protective Service',
        ' Armed-Forces': 'Armed Forces',
        ' Priv-house-serv': 'Private House Service'
    }
    return occupation_mapping.get(occupation, 'Other')

In [9]:
def generalize_occupation_level2(occupation):
    if occupation in ['Management', 'Professional', 'Clerical', 'Tech Support']:
        return 'White-Collar'
    elif occupation in ['Service', 'Sales']:
        return 'Service and Sales'
    elif occupation in ['Crafts', 'Transport', 'Machine Operation', 'Protective Service']:
        return 'Skilled Labor'
    elif occupation in ['Farming', 'Unknown', 'Armed Forces', 'Private House Service']:
        return 'Other and Unknown'
    else:
        return 'Other'

In [10]:
def generalize_occupation_level3(occupation):
    if occupation in ['White-Collar', 'Service and Sales']:
        return 'Professional and Service'
    else:
        return 'Skilled Labor and Other'

In [14]:
# Define parameters
l_value = 1  # Specify the desired l-diversity value
quasi_identifiers = ['Age', 'Education', 'Marital-Status', 'Race']  # Specify your quasi-identifiers

# Define the generalization levels for the "occupation" attribute
generalization_levels = {
    'level1': generalize_occupation_level1,
    'level2': generalize_occupation_level2,
    'level3': generalize_occupation_level3
}

# Function to achieve l-diversity
def achieve_ldiversity(df, l_value, quasi_identifiers, generalization_levels):
    while True:
        all_groups_meet_l_diversity = True

        # Group the data by quasi-identifiers
        grouped = df.groupby(quasi_identifiers)

        for group_key, data_group in grouped:
            unique_occupations = data_group['Occupation'].unique()

            if len(unique_occupations) < l_value:
                all_groups_meet_l_diversity = False

                # Find the most specific occupation level that satisfies l-diversity
                for level, generalize_func in generalization_levels.items():
                    data_group['Occupation'] = data_group['Occupation'].apply(generalize_func)
                    unique_occupations = data_group['Occupation'].unique()

                    if len(unique_occupations) >= l_value:
                        break

                # Update the original DataFrame with the anonymized data
                df.loc[data_group.index] = data_group

        if all_groups_meet_l_diversity:
            break

    return df


# Call the function to achieve l-diversity
anonymized_df = achieve_ldiversity(df, l_value, quasi_identifiers, generalization_levels)

# You can now save anonymized_df to a new CSV file or use it for further analysis.
